In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
base_dir = '../data/telemoji/pre'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
        rescale=1/255,    
#        rotation_range=40,
#        width_shift_range=0.1,
#        height_shift_range=0.1,
#        zoom_range=0.2,
#        horizontal_flip=True,
#        vertical_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
     rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    color_mode='grayscale',
    target_size=(150,150),
    batch_size=100,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    color_mode='grayscale',
    target_size=(150,150),
    batch_size=100,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
    color_mode='grayscale',
    target_size=(150,150),
    batch_size=100,
    class_mode='categorical'
)

Found 15986 images belonging to 7 classes.
Found 3992 images belonging to 7 classes.
Found 2074 images belonging to 7 classes.


# CNN

In [ ]:
### block 1 ###
# filter 1
W1 = tf.Variable(tf.random.normal([3, 3, 1, 64]))

# convolution 1
C1 = tf.nn.conv2d(train_generator, W1, 
                  strides=[1,1,1,1], padding='SAME')
C1 = tf.nn.relu(C1)

# filter 2
W2 = tf.Variable(tf.random.normal([3, 3, 64, 64]))

# convolution 2
C2 = tf.nn.conv2d(C1, W2, 
                  strides=[1,1,1,1], padding='SAME')
C2 = tf.nn.relu(C2)

# max pooling 1
P1 = tf.nn.max_pool(C2, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')

# > (?,76,76,64)

In [5]:
input_shape = (4, 28, 28, 3)

x = tf.random.normal(input_shape)
C1 = tf.keras.layers.Conv2D(
filters=2, kernel_size=3, activation='relu', input_shape=input_shape[1:]
)(x)

(4, 26, 26, 2)


In [ ]:
### block 2 ###
# filter 3
W3 = tf.Variable(tf.random.normal([3, 3, 64, 128]))

# convolution 3
C3 = tf.nn.conv2d(P1, W3, 
                  strides=[1,1,1,1], padding='SAME')
C3 = tf.nn.relu(C3)

# filter 4
W4 = tf.Variable(tf.random.normal([3, 3, 128, 128]))

# convolution 4
C4 = tf.nn.conv2d(C3, W4, 
                  strides=[1,1,1,1], padding='SAME')
C4 = tf.nn.relu(C4)

# filter 5
W5 = tf.Variable(tf.random.normal([3, 3, 128, 128]))

# convolution 5
C5 = tf.nn.conv2d(C4, W5, 
                  strides=[1,1,1,1], padding='SAME')
C5 = tf.nn.relu(C5)

# pooling 2
P2 = tf.nn.max_pool(C5, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='VALID')

# > (?,38,38,128)

# FC 1
FC1 = tf.reshape(P2, [-1, 38*38*128])

W_FC1 = tf.Variable(tf.random.normal([38*38*128, 256]))
b_FC1 = tf.Variable(tf.random.normal([256]))
FC1 = tf.nn.relu(tf.matmul(FC1, W_FC1) + b_FC1)

In [ ]:
### block 3 ###
# filter 6
W6 = tf.Variable(tf.random.normal([3, 3, 128, 256]))

# convolution 6
C6 = tf.nn.conv2d(P2, W6, 
                  strides=[1,1,1,1], padding='SAME')
C6 = tf.nn.relu(C6)

# filter 7
W7 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 7
C7 = tf.nn.conv2d(C6, W7, 
                  strides=[1,1,1,1], padding='SAME')
C7 = tf.nn.relu(C7)

# filter 8
W8 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 8
C8 = tf.nn.conv2d(C7, W8, 
                  strides=[1,1,1,1], padding='SAME')
C8 = tf.nn.relu(C8)

# filter 9
W9 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 9
C9 = tf.nn.conv2d(C8, W9, 
                  strides=[1,1,1,1], padding='SAME')
C9 = tf.nn.relu(C9)

# pooling 3
P3 = tf.nn.max_pool(C9, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')

# > (?,20,20,256)

# FC 2
FC2 = tf.reshape(P3, [-1, 20*20*256])

W_FC2 = tf.Variable(tf.random.normal([20*20*256, 256]))
b_FC2 = tf.Variable(tf.random.normal([256]))
FC2 = tf.nn.relu(tf.matmul(FC2, W_FC2) + b_FC2)

In [ ]:
### block 4 ###
# filter 10
W10 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 10
C10 = tf.nn.conv2d(P3, W10, 
                  strides=[1,1,1,1], padding='SAME')
C10 = tf.nn.relu(C10)

# filter 11
W11 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 11
C11 = tf.nn.conv2d(C10, W11, 
                  strides=[1,1,1,1], padding='SAME')
C11 = tf.nn.relu(C11)

# filter 12
W12 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 12
C12 = tf.nn.conv2d(C11, W12, 
                  strides=[1,1,1,1], padding='SAME')
C12 = tf.nn.relu(C12)

# filter 13
W13 = tf.Variable(tf.random.normal([3, 3, 256, 256]))

# convolution 13
C13 = tf.nn.conv2d(C12, W13, 
                  strides=[1,1,1,1], padding='SAME')
C13 = tf.nn.relu(C13)

# pooling 4
P4 = tf.nn.max_pool(C13, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='VALID')

# > (?,10,10,256)

# FC 3
FC3 = tf.reshape(P2, [-1, 10*10*256])

W_FC3 = tf.Variable(tf.random.normal([10*10*256, 256]))
b_FC3 = tf.Variable(tf.random.normal([256]))
FC3 = tf.nn.relu(tf.matmul(FC3, W_FC3) + b_FC3)

In [ ]:
### block 5 ###
# filter 14
W14 = tf.Variable(tf.random.normal([3, 3, 256, 512]))

# convolution 14
C14 = tf.nn.conv2d(P4, W14, 
                  strides=[1,1,1,1], padding='SAME')
C14 = tf.nn.relu(C14)

# filter 15
W15 = tf.Variable(tf.random.normal([3, 3, 512, 512]))

# convolution 15
C15 = tf.nn.conv2d(C14, W15, 
                  strides=[1,1,1,1], padding='SAME')
C15 = tf.nn.relu(C15)

# filter 16
W16 = tf.Variable(tf.random.normal([3, 3, 512, 512]))

# convolution 16
C16 = tf.nn.conv2d(C15, W16, 
                  strides=[1,1,1,1], padding='SAME')
C16 = tf.nn.relu(C16)

# filter 17
W17 = tf.Variable(tf.random.normal([3, 3, 512, 512]))

# convolution 17
C17 = tf.nn.conv2d(C16, W17, 
                  strides=[1,1,1,1], padding='SAME')
C17 = tf.nn.relu(C17)

# pooling 5
P5 = tf.nn.max_pool(C17, ksize=[1, 2, 2, 1], 
                    strides=[1, 2, 2, 1], padding='SAME')

# > (?,6,6,512)


# flatten
# DNN으로 들어가야할 것은 한개의 이미지에 대한 데이터
P5 = tf.reshape(P5, [-1, 6*6*512])

# concat
P5 = tf.concat([FC1, FC2, FC3, P5], 0)

W18 = tf.Variable(tf.random.normal([(38*38*128)+(20*20*256)+(10*10*256)+(6*6*512), 7]))
b18 = tf.Variable(tf.random.normal([7]))

# hypothesis
logit = tf.matmul(P5, W18) + b18
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)